<a href="https://colab.research.google.com/github/vsattamini/silvertone/blob/luiz/model_mel_fourrier_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tensorflow.keras import layers, Sequential, Input, Model
from  tensorflow import expand_dims
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils, to_categorical
from sklearn.preprocessing import MinMaxScaler
import torch
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_pickle("/content/drive/MyDrive/df/database_final_v4_6emo.pkl")

In [3]:
y = df["emotion"]
X_mel = np.array(df["mel_spec"].values.tolist()).astype('float32')

In [4]:
X_fourrier = np.array(df["fourrier"].values.tolist()).astype('float32')

In [5]:
X_sep = np.array(df["spectral_centroid"].values.tolist()).astype('float32')

In [6]:
X_sep = X_sep.reshape(11770,127)

In [7]:
X_mel = expand_dims(X_mel, -1)
X_fourrier = expand_dims(X_fourrier, -1)

In [8]:
lb = LabelEncoder()
y= np_utils.to_categorical(lb.fit_transform(y))

In [9]:
input_mel = Input(shape=(X_mel.shape[1:]))
input_sep = Input(shape=(X_sep.shape[1:]))
input_fourrier = Input(shape=(X_fourrier.shape[1:]))

In [10]:
mel = layers.Conv2D(64, (4,4), activation="relu", padding="same")(input_mel)
mel = layers.BatchNormalization()(mel)
mel = layers.MaxPool2D((2,2))(mel)
mel = layers.Dropout(rate=0.2)(mel)
mel = layers.Conv2D(128, (4,4), activation="relu", padding="same")(mel)
mel = layers.Flatten()(mel)
mel = Model(inputs=input_mel, outputs=mel)

In [11]:
fourrier = layers.Conv2D(64, (4,4), activation="relu", padding="same")(input_fourrier)
fourrier = layers.BatchNormalization()(fourrier)
fourrier = layers.MaxPool2D((2,2))(fourrier)
fourrier = layers.Dropout(rate=0.2)(fourrier)
fourrier = layers.Conv2D(128, (4,4), activation="relu", padding="same")(fourrier)
fourrier = layers.Flatten()(fourrier)
fourrier = Model(inputs=input_fourrier, outputs=fourrier)

In [12]:
sep = layers.Dense(128, activation="relu")(input_sep)
sep = layers.BatchNormalization()(sep)
sep = layers.Dropout(rate=0.2)(sep)
sep = Model(inputs=input_sep, outputs=sep)

In [13]:
combined = layers.Concatenate()([mel.output, fourrier.output, sep.output])

In [14]:
final = layers.Dense(10, activation="relu")(combined)
final = layers.Dense(128, activation="relu")(final)
final = layers.BatchNormalization()(final)
final = layers.Dropout(rate=0.2)(final)
final = layers.Dense(7, activation="softmax")(final)

In [15]:
model = Model(inputs=[mel.input, fourrier.input, sep.input], outputs=final)

In [16]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(x=[X_mel, X_fourrier, X_sep], y = y, validation_split=0.3, epochs=100, batch_size=8, callbacks=[es])

Epoch 1/500
515/515 [==============================] - 1067s 2s/step - loss: 1.5760 - accuracy: 0.4141 - val_loss: 1.8224 - val_accuracy: 0.3557
Epoch 2/500
515/515 [==============================] - 1059s 2s/step - loss: 1.3227 - accuracy: 0.5186 - val_loss: 1.6039 - val_accuracy: 0.3925
Epoch 3/500
515/515 [==============================] - 1066s 2s/step - loss: 1.1016 - accuracy: 0.6013 - val_loss: 1.5279 - val_accuracy: 0.4359
Epoch 4/500
515/515 [==============================] - 1065s 2s/step - loss: 0.9705 - accuracy: 0.6537 - val_loss: 1.6547 - val_accuracy: 0.3857
Epoch 5/500
515/515 [==============================] - 1064s 2s/step - loss: 0.8405 - accuracy: 0.7006 - val_loss: 1.6502 - val_accuracy: 0.4084
Epoch 6/500
515/515 [==============================] - 1065s 2s/step - loss: 0.6930 - accuracy: 0.7542 - val_loss: 1.7561 - val_accuracy: 0.4208
Epoch 7/500
515/515 [==============================] - 1062s 2s/step - loss: 0.5249 - accuracy: 0.8233 - val_loss: 1.7753 - val_ac